In [1]:
import asyncio
import neaspec
import configuration as config

await neaspec.connect(config.db, config.vm, config.dll)

from neaspec import context

> Redirecting unknown loading assembly System.Runtime.CompilerServices.Unsafe, Version=4.0.4.1, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a
> Configuration in local file C:\Users\Patrick\AppData\Local\Python_Software_Foundatio\DefaultDomain_Path_mndturxegrmy5qhqcgfnwau3k5cajhfi\3.7.7\user.config
> Loaded from database group: Nea.Client.Hardware.Microscope.Properties.Settings
> Loaded from database group: Nea.Client.Hardware.Camera.Properties.Settings
> No camera found. Discovery result: Success
> Waiting for initialization...
> Trying to connect to 127.0.0.1
> Trying to connect to 127.0.0.1
> Trying to connect to 127.0.0.1
> Connected to 127.0.0.1
> Connected to 127.0.0.1
> Connected to 127.0.0.1
> Receive thread (Nea.Client.Hardware.PreviewRpc) started.
> Receive thread (Nea.Client.Hardware.MotorRpc) started.
> Receive thread (Nea.Client.Hardware.Rpc) started.
> Client 1.10.7395 is incompatible with Server 1.10.7397.
> Client 1.10.7395 is incompatible with Server 1.10.7397.
> Ser

# DemodulationMode

The `DemodulationMode` specifies how to demodulate the signals from the detectors.

In [2]:
import Nea.Client.SharedDefinitions as nea
import System

# shows all pre-installed demodulation modes 
print(*System.Enum.GetNames(nea.DemodulationMode))

None PsHet Plugin Fourier TipForce KelvinProbeAdvanced PhotoThermalExpansionAdvanced AdvancedSoftMatterAdvanced ThermalAnalysis Storyboard ClassicalPhotoThermalExpansion PhotoThermalExpansion PluginAdvanced KelvinProbe ContactMode ThermalAnalysisAdvanced AdvancedSoftMatter PiezoResponse


In [7]:
# reads which mode is currently active
mode = context.Microscope.DemodulationMode

# displays the numerical identifier (64-bit)
print(hex(mode))

# translates the id of a pre-installed mode to a human readable string or "None" for all others
print(System.Enum.GetName(nea.DemodulationMode, System.UInt64(mode)))

# changes the demodulation mode
context.Microscope.ChangeDemodulationMode.Execute(nea.DemodulationMode.Fourier)

0xd0a252eb31f5
Fourier


## Parameters

Some modes provide additional parameters: They can be accessed by name through the dictionary `DemodulationModeParameters`:

In [4]:
# Show all parameters in the current demodulation mode 
print(*context.Microscope.DemodulationModeParameters.Keys)
print([parameter.Value for parameter in context.Microscope.DemodulationModeParameters.Values])

stiffness = context.Microscope.DemodulationModeParameters["CantileverStiffness"]
print(stiffness)

# the object contains also metadata about the parameter
print(f'MIN/MAX: {stiffness.Range}')

# changes the value and blocks until done
stiffness.ChangeValue(0.1123).Wait()

# shows the last known (cached) value
print(stiffness.Value)

# refreshes and blocks until done
stiffness.Refresh().Wait()

# refreshes the value and blocks
print(stiffness.GetCurrentValue().Result)

#async def setParameterValue(parameter, value): return parameter.ChangeValue(value).Result
#await asyncio.create_task(setParameterValueAsync(stiffness, 0.005))

QualityFactor CantileverStiffness CantileverLength CantileverWidth CurrentWavelength
[nan, 0.01624513105859481, 5.499999999999998, 2.8000000000000003, nan]
CantileverStiffness: 0,0162451310585948 (double)
MIN/MAX: 2.2250738585072E-308 1.79769313486232E+308
0.1123
0.1123


   ## Create a customized demodulation mode (Storyboard)
   
   You can create a JSON string that defines the characteristics of the demodulation (*Storyboard*) and store it in the database. You can keep its id and reuse it the next time.

In [3]:
import json

storyboard = {
    "Inputs" : {
        # Possible options:
        # "NoDataAcquisition", "Daq_1024_2500ksps", "Daq_2048_5Msps", "Daq_4096_10Msps", 
        "Mechanical" : "Daq_2048_5Msps",
        "Optical"    : "Daq_1024_2500ksps",
        "A"          : "Daq_1024_2500ksps",
        "B"          : "Daq_1024_2500ksps",
    },
    "Channels" : {
        "Mechanical" : {
            0 : 0.0,  # DC
            1 : context.Microscope.HeadFrequency, 
            2 : context.Microscope.HeadFrequency * 2.0, 
        }
    }
}

insertTask = context.Database.InsertStoryboard(json.dumps(storyboard), "My Test")


arguments = nea.DemodulationModeArguments()
arguments.Add("StoryboardId", insertTask.Result)

setModeTask = context.Microscope.SetDemodulationModeAsync("Storyboard", arguments)
setModeTask.Wait()



In [5]:
context.Dispose()

> Disposing...
> Dispose RpcPreview (001)
> Preview ending.
> Send thread (Nea.Client.Hardware.PreviewRpc) stopped.
> Receive thread (Nea.Client.Hardware.PreviewRpc) stopped.
> Send thread (Nea.Client.Hardware.MotorRpc) stopped.
> Receive thread (Nea.Client.Hardware.MotorRpc) stopped.
> Send thread (Nea.Client.Hardware.Rpc) stopped.
> Receive thread (Nea.Client.Hardware.Rpc) stopped.
